<a href="https://colab.research.google.com/github/LatiefDataVisionary/text-mining-and-natural-language-processing-college-task/blob/main/notebooks/decison_tree_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import Libary**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
    ConfusionMatrixDisplay
)
import matplotlib.pyplot as plt

**1. Load Dataset**

In [12]:
path = 'https://raw.githubusercontent.com/LatiefDataVisionary/text-mining-and-natural-language-processing-college-task/refs/heads/main/datasets/ramadan_labeled_sentiment.csv'
df = pd.read_csv(path)
df

,tweet_clean,Tweet,sentiment,sentiment_scores,neg,neu,pos,compound
0,"['abraj', 'al', 'bait', 'clock', 'tower', 'bea...",abraj al bait clock tower beams indicating com...,negative,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
1,"['accounts', 'recognised', 'ramadan', 'none', ...",accounts recognised ramadan none recognised be...,negative,"{'neg': 0.147, 'neu': 0.853, 'pos': 0.0, 'comp...",0.147,0.853,0.000,-0.4767
2,"['admin', 'post', 'peaceful', 'ramadan', 'cele...",admin post peaceful ramadan celebrations east ...,positive,"{'neg': 0.0, 'neu': 0.714, 'pos': 0.286, 'comp...",0.000,0.714,0.286,0.4939
3,"['admin', 'post', 'ramadan', 'norway']",admin post ramadan norway,negative,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
4,"['admin', 'post', 'ramadan', 'usual', 'peacefu...",admin post ramadan usual peaceful start englan...,positive,"{'neg': 0.0, 'neu': 0.775, 'pos': 0.225, 'comp...",0.000,0.775,0.225,0.4939
...,...,...,...,...,...,...,...,...
831,"['ya', 'allah', 'please', 'remove', 'procrasti...",ya allah please remove procrastination body ra...,positive,"{'neg': 0.171, 'neu': 0.348, 'pos': 0.481, 'co...",0.171,0.348,0.481,0.8689
832,"['yaqeen', 'launched', 'official', 'urdu', 'yo...",yaqeen launched official urdu youtube channel ...,positive,"{'neg': 0.0, 'neu': 0.688, 'pos': 0.312, 'comp...",0.000,0.688,0.312,0.7003
833,"['yasser', 'abu', 'daqqa', 'recently', 'freed'...",yasser abu daqqa recently freed palestinian de...,negative,"{'neg': 0.336, 'neu': 0.494, 'pos': 0.17, 'com...",0.336,0.494,0.170,-0.7506
834,"['young', 'arabs', 'took', 'streets', 'last', ...",young arabs took streets last night shouting a...,negative,"{'neg': 0.19, 'neu': 0.747, 'pos': 0.063, 'com...",0.190,0.747,0.063,-0.6597


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 836 entries, 0 to 835
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tweet_clean       836 non-null    object 
 1   Tweet             836 non-null    object 
 2   sentiment         836 non-null    object 
 3   sentiment_scores  836 non-null    object 
 4   neg               836 non-null    float64
 5   neu               836 non-null    float64
 6   pos               836 non-null    float64
 7   compound          836 non-null    float64
dtypes: float64(4), object(4)
memory usage: 52.4+ KB


**2. Data Preprocessing**

Konversi label ke numerik


In [9]:
df['sentiment'] = df['sentiment'].map({'positi': 1, 'negative' : 0})

Handle missing values

In [14]:
print('Missing values:\n', df.isnull().sum())

Missing values:
 tweet_clean         0
Tweet               0
sentiment           0
sentiment_scores    0
neg                 0
neu                 0
pos                 0
compound            0
dtype: int64


In [16]:
df = df.dropna(subset=['tweet_clean', 'sentiment'])
print('\nMissing values after treatment:\n', df.isnull().sum())


Missing values after treatment:
 tweet_clean         0
Tweet               0
sentiment           0
sentiment_scores    0
neg                 0
neu                 0
pos                 0
compound            0
dtype: int64


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 836 entries, 0 to 835
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tweet_clean       836 non-null    object 
 1   Tweet             836 non-null    object 
 2   sentiment         836 non-null    object 
 3   sentiment_scores  836 non-null    object 
 4   neg               836 non-null    float64
 5   neu               836 non-null    float64
 6   pos               836 non-null    float64
 7   compound          836 non-null    float64
dtypes: float64(4), object(4)
memory usage: 52.4+ KB


**3. Feature Engineering**

Gabungkan list kata menjadi string

In [19]:
df['text_processed'] = df['tweet_clean'].apply(
    lambda x: ' '.join(eval(x)) if isinstance(x, str) else ''
)